
### Done
* Forward pass


### Missing
* preprocessing
* Backwards pass
* Summaries
* Replay buffer
* target network
* Save model
* Breakout 
* Let it train

### Notes
Inspired by: https://medium.com/@awjuliani/simple-reinforcement-learning-with-tensorflow-part-4-deep-q-networks-and-beyond-8438a3e2b8df

# DQN on Breakout



In [1]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }
});

<IPython.core.display.Javascript object>

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib inline

In [3]:
import numpy as np
import tensorflow as tf
import gym

from tensorflow.contrib.keras.api.keras.layers import Dense, Input, Conv2D, Flatten
from tensorflow.contrib.keras.api.keras.models import Model

import utils

In [4]:
## Hyperparameters

batch_size = 32
training_time = int(50e6) # frames
training_time = int(1e5) # frames

replay_buffer_size = int(1e6)
replay_buffer_size = int(1e4)

learning_rate = 0.00025
gamma = 0.99
epsilon_init = 1.0
epsilon_end = 0.1
annealing_period = 1e6 # frames
annealing_period = 1e5 # frames


In [5]:
## Parameters
env_name = 'Breakout-v0'
env_name = 'CartPole-v1'



In [6]:
## Derived settings
annealer = utils.Annealer(epsilon_init, epsilon_end, annealing_period)

env = gym.make(env_name)
obs = env.reset()
s_size = list(obs.shape)
a_size = env.action_space.n

obsPlaceholder = tf.placeholder(tf.float32, shape=[None]+s_size, name='obsPlaceholder')
# obsPlaceholder = tf.placeholder(tf.float32, shape=[None]+[84, 84, 4], name='obsPlaceholder')
# Assume action is encoded as ONE number
actionPlaceholder = tf.placeholder(tf.int32, shape=[None], name='actionPlaceholder')
rewardPlaceholder = tf.placeholder(tf.float32, shape=[None], name='rewardPlaceholder')
targetQPlaceholder = tf.placeholder(tf.float32, shape=[None], name='targetQPlaceholder')

print('-- Placeholders --')
print('obsPlaceholder', obsPlaceholder.get_shape())
print('actionPlaceholder', actionPlaceholder.get_shape())
print('rewardPlaceholder', rewardPlaceholder.get_shape())
print('targetQPlaceholder', targetQPlaceholder.get_shape())
print()

### Create model
from models import DQN
with tf.name_scope('DQN1'):
    DQN1 = DQN('dense', obsPlaceholder, actionPlaceholder, a_size)
    DQN1.create_train_op(targetQPlaceholder, learning_rate=learning_rate)

# print("-- Internal tensors --")
# print('qvalues', qvalues.get_shape(), 'One for each action')
# print('selectAction', selectAction.get_shape(), 'Just 1')
# print('qvalue', qvalue.get_shape(), 'Just 1')



[2017-06-14 13:58:13,901] Making new env: CartPole-v1


-- Placeholders --
obsPlaceholder (?, 4)
actionPlaceholder (?,)
rewardPlaceholder (?,)
targetQPlaceholder (?,)



In [7]:
# utils.Experience_buffer().test()

In [ ]:
def new_episode(env):
    """Simple wrapper that restarts the environment"""
    obs = env.reset()
    episode_time_step = 0
    episode_reward = 0
    return obs, episode_time_step, episode_reward

In [ ]:
update_frequency = 32 
assert update_frequency > 1, "The dimensions doen't allign if update_frequency" \
    + 'is less than 1: ' + str(update_frequency)
minimum_experience = int(1e4)

buffer = utils.Experience_buffer(replay_buffer_size)

try:
    sess.close()
except:
    pass
sess = tf.Session()
sess.run(tf.global_variables_initializer())

training_summaries = {}
training_summaries['num_ep'] = 0
training_summaries['ep_rewards'] = []
training_summaries['epsilon'] = []

print('{:>9}  {:>7}  {:>7}  {:>7}'.format('frames', 'epis', 'reward', 'epsilon'))

obs, ep_t, ep_r = new_episode(env)
f = 0
for f in range(training_time):
    try:
        epsilon = annealer.linear(f)
    #     action_chosen = sess.run(action, feed_dict={obsPlaceholder : [obs]}) if np.random.rand(1) > epsilon \
    #         else np.random.randint(a_size)
        [action] = sess.run(DQN1.action, feed_dict={obsPlaceholder : [obs]})
#         print('action_chosen', type(action_chosen), action_chosen)
        assert 'int' in str(type(action))
        next_obs, reward, done, _ = env.step(action)
        # TODO: Preprocess obs
        # TODO: wrap in action repeater
        ep_t += 1
        ep_r += reward
        reward = -1 if done and ep_t < 400 else reward

        experience = {'obs':[obs], 'action':action, 'reward':reward,
                      'next_obs':[next_obs], 'done':done}
        buffer.add(experience)
        obs = next_obs
        
        ## Update Weights
        if f % update_frequency == 0 \
            and f > minimum_experience:
            train_batch = buffer.sample(batch_size)

            # Compute ... TODO: add description
            action, actualQs = sess.run([DQN1.action, DQN1.Qout],
                feed_dict={obsPlaceholder : train_batch['next_obs']}
                 )

            actualQ = actualQs[range(batch_size), action] # The Q value of the chosen action
            not_done = train_batch['done']*(-1) + 1 # Used to remove the actualQ
                                                    # When at terminal state
            target = train_batch['reward'] + gamma*actualQ*not_done

    #         print("train_batch['obs']")
    #         print(type(train_batch['obs']))
    #         print(train_batch['obs'].shape)
    #         print()

    #         print('Q1')
    #         print(type(Q1))
    #         print(Q1.shape)
    #         print()

    #         print("train_batch['action']")
    #         print(type(train_batch['action']))
    #         print(train_batch['action'].shape)
    #         print()

    #         print("target")
    #         print(type(target))
    #         print(target.shape)
    #         print()

            Q1_dict = {obsPlaceholder : train_batch['obs'],
                       actionPlaceholder : train_batch['action'],
                       targetQPlaceholder : target # TODO: Should be target network!
                }
            _ = sess.run(DQN1.train_op, feed_dict=Q1_dict)
        if f % 10000 == 0 and f>0:
            print('{:9}, {:7}, {:7.1f}, {:7.3f}'.format(
                  f, 
                  training_summaries['num_ep'],
                  np.mean(training_summaries['ep_rewards']),
                  training_summaries['epsilon'][-1],
                 ))
        if done:
            training_summaries['num_ep'] += 1
            training_summaries['ep_rewards'].append(ep_t)
            training_summaries['epsilon'].append(epsilon)
            obs, ep_t, ep_r = new_episode(env)
    except KeyboardInterrupt:
        break
        

# env.render(close=True)
print('\nTerminated')

   frames     epis   reward  epsilon
    10000,     743,    13.4,   0.900
    20000,    1808,    11.1,   0.800
    30000,    2879,    10.4,   0.700
    40000,    3944,    10.1,   0.600
    50000,    5016,    10.0,   0.500
    60000,    6087,     9.9,   0.400


In [ ]:
training_summaries['epsilon'][-1]

In [ ]:
## Run and render a forward pass

obs = env.reset()
while True:
    try:
        [action_chosen] = sess.run(selectAction, feed_dict={obsPlaceholder : [obs]})
        assert 'int' in str(type(action_chosen))
        obs, reward, done, _ = env.step(action_chosen)
        env.render()

        if done:
            obs = env.reset()
    except:
        break
        
print('Terminated')

In [ ]:
env.render(close=True)